# Training Analysis - Italic Detection Model

This notebook analyzes the training process and visualizes model performance metrics.

## Setup and Imports

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add parent directory to path
sys.path.append('..')
import src.config as config

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print("✅ Imports successful")

## Load Training Metrics

In [ ]:
# Load training metrics
metrics_path = config.MODEL_DIR / 'indobert-italic' / 'training_metrics.json'

if metrics_path.exists():
    with open(metrics_path, 'r') as f:
        training_metrics = json.load(f)
    print("✅ Training metrics loaded")
    print(f"Available metrics: {list(training_metrics.keys())}")
else:
    print("❌ Training metrics not found. Run training first.")
    print(f"Expected path: {metrics_path}")

In [ ]:
# Display final metrics
if 'training_metrics' in locals():
    print("Final Validation Metrics:")
    print("="*50)
    for key, value in training_metrics.items():
        if isinstance(value, float):
            print(f"{key:20s}: {value:.4f}")
        else:
            print(f"{key:20s}: {value}")

## Load TensorBoard Logs

Note: This requires TensorBoard event files in the logs directory.

In [ ]:
# Try to load TensorBoard logs
try:
    from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
    
    log_dir = config.LOG_DIR
    
    # Find event files
    event_files = list(log_dir.rglob('events.out.tfevents.*'))
    
    if event_files:
        print(f"✅ Found {len(event_files)} TensorBoard event file(s)")
        
        # Load the most recent one
        latest_event = sorted(event_files)[-1]
        print(f"Loading: {latest_event}")
        
        event_acc = EventAccumulator(str(latest_event.parent))
        event_acc.Reload()
        
        print(f"Available tags: {event_acc.Tags()}")
    else:
        print("⚠️  No TensorBoard event files found")
        event_acc = None
        
except ImportError:
    print("⚠️  TensorBoard not installed. Install with: pip install tensorboard")
    event_acc = None
except Exception as e:
    print(f"⚠️  Error loading TensorBoard logs: {e}")
    event_acc = None

## Visualize Training Curves

In [ ]:
if event_acc is not None:
    # Extract training loss
    try:
        train_loss = event_acc.Scalars('train/loss')
        steps_train = [x.step for x in train_loss]
        values_train = [x.value for x in train_loss]
        
        # Plot training loss
        plt.figure(figsize=(14, 6))
        plt.plot(steps_train, values_train, label='Training Loss', color='blue', alpha=0.7)
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.title('Training Loss Curve', fontsize=16, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    except:
        print("⚠️  Training loss curve not available")
else:
    print("⚠️  Cannot plot training curves without TensorBoard logs")

In [ ]:
if event_acc is not None:
    # Extract validation metrics
    try:
        eval_metrics = {}
        for metric in ['eval/loss', 'eval/accuracy', 'eval/precision', 'eval/recall', 'eval/f1']:
            try:
                data = event_acc.Scalars(metric)
                eval_metrics[metric] = {
                    'steps': [x.step for x in data],
                    'values': [x.value for x in data]
                }
            except:
                pass
        
        if eval_metrics:
            # Plot all metrics
            fig, axes = plt.subplots(2, 2, figsize=(16, 10))
            axes = axes.flatten()
            
            metric_names = ['eval/accuracy', 'eval/precision', 'eval/recall', 'eval/f1']
            titles = ['Validation Accuracy', 'Validation Precision', 'Validation Recall', 'Validation F1 Score']
            colors = ['green', 'orange', 'purple', 'red']
            
            for i, (metric, title, color) in enumerate(zip(metric_names, titles, colors)):
                if metric in eval_metrics:
                    axes[i].plot(eval_metrics[metric]['steps'], 
                               eval_metrics[metric]['values'], 
                               color=color, linewidth=2, marker='o')
                    axes[i].set_xlabel('Steps')
                    axes[i].set_ylabel(title.split()[1])
                    axes[i].set_title(title, fontweight='bold')
                    axes[i].grid(True, alpha=0.3)
                    
                    # Add final value annotation
                    final_val = eval_metrics[metric]['values'][-1]
                    axes[i].annotate(f'Final: {final_val:.4f}',
                                   xy=(eval_metrics[metric]['steps'][-1], final_val),
                                   xytext=(10, 10), textcoords='offset points',
                                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                                   fontweight='bold')
            
            plt.tight_layout()
            plt.show()
        else:
            print("⚠️  No evaluation metrics found in TensorBoard logs")
    except Exception as e:
        print(f"⚠️  Error plotting validation metrics: {e}")
else:
    print("⚠️  Cannot plot validation curves without TensorBoard logs")

## Load Test Results

In [ ]:
# Load test results
test_results_path = config.MODEL_DIR / 'indobert-italic' / 'test_results.json'

if test_results_path.exists():
    with open(test_results_path, 'r') as f:
        test_results = json.load(f)
    print("✅ Test results loaded")
    
    # Display classification report
    if 'classification_report' in test_results:
        print("\nTest Set Classification Report:")
        print("="*80)
        print(test_results['classification_report'])
else:
    print("❌ Test results not found. Run evaluation first.")
    test_results = None

## Visualize Confusion Matrix

In [ ]:
if test_results is not None and 'confusion_matrix' in test_results:
    cm = np.array(test_results['confusion_matrix'])
    labels = ['O', 'B', 'I']
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels,
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted Label', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=14, fontweight='bold')
    plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Calculate and display per-class accuracy
    print("\nPer-Class Accuracy:")
    print("="*50)
    for i, label in enumerate(labels):
        accuracy = cm[i, i] / cm[i].sum()
        print(f"{label:5s}: {accuracy:.4f} ({accuracy*100:.2f}%)")
else:
    print("⚠️  Confusion matrix not available")

## Model Performance Summary

In [ ]:
# Create summary dataframe
if 'training_metrics' in locals():
    summary_data = {
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'Validation': [
            training_metrics.get('eval_accuracy', 0),
            training_metrics.get('eval_precision', 0),
            training_metrics.get('eval_recall', 0),
            training_metrics.get('eval_f1', 0)
        ]
    }
    
    df_summary = pd.DataFrame(summary_data)
    
    print("\nModel Performance Summary:")
    print("="*50)
    print(df_summary.to_string(index=False))
    
    # Visualize
    plt.figure(figsize=(10, 6))
    x = range(len(df_summary))
    plt.bar(x, df_summary['Validation'], color='steelblue', alpha=0.7, edgecolor='black')
    plt.xticks(x, df_summary['Metric'])
    plt.ylabel('Score', fontsize=12)
    plt.title('Model Performance Metrics (Validation Set)', fontsize=16, fontweight='bold')
    plt.ylim(0, 1.0)
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, v in enumerate(df_summary['Validation']):
        plt.text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

## Save Visualizations

In [ ]:
# Create output directory for plots
output_dir = config.MODEL_DIR / 'training_visualizations'
output_dir.mkdir(exist_ok=True)

print(f"Visualizations can be saved to: {output_dir}")
print("\nTo save plots, add plt.savefig() before plt.show() in cells above.")

## Recommendations

Based on the training analysis:

1. **Model Convergence**: Check if the training loss has plateaued
2. **Overfitting**: Compare training vs validation metrics
3. **Class Imbalance**: Review confusion matrix for class-specific issues
4. **Early Stopping**: Verify if early stopping triggered appropriately

For production deployment:
- Ensure F1 score > 0.95 for reliable predictions
- Monitor false positive/negative rates
- Consider ensemble methods if single model performance insufficient